# Write A Poker Program 

## Notation
Input: 5 cards: Every card has a rank and a suit  
example: the 5 of diamonds, the rank is 5 and the suit is diamond  
Output: return the best hand  
four suit: Spades(S) Heart(H) Diamonds(D) Club(C)

## The rule of poker:
hand_rank -> map a hand to something  
example:  
j j 2 2 5 -> 2 pairs  
5 6 7 8 9 -> straight  
10 9 5 4 2 -> flush (the same suit)  

### Q1:Representing hands  
(1)\['JS', 'JD', '2S', '2C', '7H'\]  
(2)\[(11, 'S'), (11, 'D'), (2, 'S'), (2,'C'), (7, 'H)\]  
(3)set(\['JS', 'JD', '2S', '2C', '7H'\])  
(4)"JS JD 2S 2C 7H"  
  
### Answer:
(3)won't work well when we have two decks  
(4) need to do some proprocess

In [8]:
"""
# Max is a function that takes a list as input and return s the highest ranking one.
# Program One:
"""
def poker(hands):
    "Return the best hand: poker([hand, ...]) => hand"
    return max(hands, key = hand_rank)

# takes a hand as input and return some sort of a rank
"""
A -> 14(AD AS AH AC 5C)
A -> 1 (AS 2S 3S 4S 5S)
straight and flush:8 TD 9D 8D 7D 6D --------> (8, 10)
4 kind:            7 9D 8S 9H 9C 6S --------> (1) 70905 (2)7.0905 (3) (7,9,5)
                      3D 3S 3H 3C 4S --------> (1) 70304 (2)7.0304 (3) (7,3,4)
full house:        6 8S 8D 8H 2S 2H --------> (6,8,2)
flush:             5 TD 8D 7D 5D 3D --------> (5,[10,8,7,5,3])
straight:          4 JC TS 9D 8C 7C --------> (4,11)
three:             3 7H 7D 7C 5C 2C --------> (3,7,[7,7,7,5,2])
two pairs:         2 JD JC 3S 3H KH --------> (2,(11,3),[13,11,11,3,3])
one pair:          1 2D 2C JD 6H 3C --------> (1,2,[11,6,3,2,2])
other:             0 7C 5C 4C 3C 2D --------> (0,7,5,4,3,2)
"""
# straight(ranks): returns True if the hand is a straight
# flush(hand):     returns True if the hand is a flush
# kind(n, rans):   return the first rank that the hand has exactly n of. For a hand with 4 sevens, this
#                  function would return 7
# two_pair(ranks): If there is a two pair, this function returns their corresponding ranks as a tuple.
#                  For example, a hand with 2 twos and 2 fours would cause this function to return (4,2)
# card_ranks(hand) return and ORDERED tuple of the ranks in a hand (where the order goes from highest to 
#                  lowerese rank)

def card_ranks(hand):
    "Return a list of ranks, sorted woth higher first"
    ranks = ['--23456789TJQKA'.index(r) for r,s in hand]
    ranks.sort(reverse = True)
    return [5, 4, 3, 2, 1] if (ranks == [14, 5, 4, 3, 2]) else ranks

def straight(ranks):
    "Return True if the ordered ranks form a 5-card straight"
    return max(ranks) - min(ranks) == 4 and len(set(ranks)) == 5
    
def flush(hand):
    "Return True if all the card have the same suit"
    suits = [s for r,s in hand]
    return len(set(suits)) == 1

def kind(n ,ranks):
    """Return the first rank that this hand has exactly n of.
    Return None if there is no n-of-a-kind in the hand."""
    for r in ranks:
        if ranks.count(r) == n:
            return r
    return None

def two_pair(ranks):
    """If there are two pair, return the two ranks as a tuple:
    (highest, lowest); otherwise return None."""
    pair = kind(2, ranks)
    lowpair = kind(2, list(reversed(ranks)))
    if pair and pair != lowpair:
        return  pair,lowpair
    return None
    
def hand_ranks(hand):
    "Return a value indicating the ranking of a hand"
    #extract the ranks of the hand
    ranks = card_ranks(hand)
    #straight and flush
    if straight(ranks) and flush(hand):
        # 2 3 4 5 6 => (8, 6) | 6 7 8 9 T => (8, 10)
        return (8, max(ranks))
    elif kind(4, ranks):
        # 9 9 9 9 3 =>(7, 9, 3)
        return (7, kind(4, ranks), kind(1, ranks))
    elif kind(3,ranks) and kind(2,ranks):
        # 8 8 8 7 7 => (6, 8, 7)
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):
        return (5, ranks)
    elif straight(ranks):
        return (4, max(ranks))
    elif kind(3, ranks):
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):
        return (2, two_pair(ranks), ranks)
    elif kind(2, ranks):
        return (1, kind(2,ranks), ranks)
    else:
        return (0, ranks)
"""
One important principle of testing is to do extreme values
"""
def test():
    "Test cases for the function in poker program"
    # straight flush
    sf = "6C 7C 8C 9C TC".split()
    # four kind
    fk = "9D 9H 9S 9C 7D".split()
    # full house
    fh = "TD TC TH 7C 7D".split()
    # two pairs
    tp = "5S 5D 9H 9C 6S".split()
    fkranks = card_ranks(fk)
    tpranks = card_ranks(tp)
    
    # add more test
    # A-5 straight
    s1 = "AS 2S 3S 4S 5C".split()
    # 2-6 straight
    s2 = "2C 3C 4C 5S 6S".split()
    # A high
    ah = "AS 2S 3S 4s 6C".split()
    # 7 high
    sh = "2S 3S 4S 6C 7D".split()
    
    #assert kind(n, ranks)
    assert kind(4, fkranks) == 9
    assert kind(3, fkranks) == None
    assert kind(2, fkranks) == None
    assert kind(1, fkranks) == 7
    
    #assert two_pair(ranks)
    assert two_pair(fkranks) == None
    assert two_pair(tpranks) == (9,5)
    
    #assert paker(hands)
    assert poker([sf, fk, fh]) == sf
    assert poker([fk, fh]) == fk
    assert poker([fh, fh]) == fh
    assert poker([s1, s2, ah, sh]) == s2
    
    #a single hand
    assert poker([sf]) == sf
    assert poker([fk]) == fk
    assert poker([fh]) == fh
    
    # test 100 hands
    assert poker([sf] + 99 * [fh]) == sf
    
    #assert hand_rank
    assert hand_ranks(sf) == (8, 10)
    assert hand_ranks(fk) == (7, 9, 7)
    assert hand_ranks(fh) == (6, 10, 7)
    
    #assert card_rank
    assert card_ranks(sf) == [10, 9, 8, 7, 6]
    assert card_ranks(fk) == [9, 9, 9, 9, 7]
    assert card_ranks(fh) == [10, 10, 10, 7, 7]

    #assert straight
    assert straight([9, 8, 7, 6, 5]) == True
    assert straight([9, 8, 8, 6 ,5]) == False
    
    #assert flush
    assert flush(sf) == True
    assert flush(fk) == False
    
    return "test pass"


print(test())

test pass


### handle ties